<a href="https://colab.research.google.com/github/ancestor9/2025_Winter_Deep-Learning-with-TensorFlow/blob/main/20260107_03_Image%20Classification/14_0_Figure_8_4_How_convolution_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Figure 8.4: How convolution works**

## **그림 단계별 설명 (화살표부분만 개념 해부)**

### (1) Input feature map
- **형태**  
$$
(\text{Input depth},\ \text{Height},\ \text{Width})
$$

- **의미**
  - Height × Width: 공간 축 (spatial dimension)
  - Input depth: 채널 수  
    - 예: RGB 이미지 → 3  
    - 중간 feature map → 더 큰 값

- **핵심**
  - 그림에서는 여러 채널을 가진 **3차원 텐서**를 한 번에 처리한다.

---

### (2) 3×3 input patches (슬라이딩 윈도우)
- 커널의 공간 크기가 **3×3**일 때,
- 입력 feature map 위를 슬라이딩하며 각 위치에서 패치를 추출한다.

- **패치 크기**
$$
(\text{Input depth},\ 3,\ 3)
$$

- **중요한 점**
  - 하나의 patch는 **이미 depth 전체를 포함**한다.(CNN에서 하나의 patch는 2D 조각이 아니라, 채널 방향까지 포함한 “작은 3D 블록”이다.)
  - 즉, 공간적으로는 3×3이지만 채널 방향으로는 입력 전체를 본다.

---

### (3) Dot product with kernel
- 하나의 필터(kernel)의 형태:
$$
(\text{Input depth},\ 3,\ 3)
$$

- **연산 과정**
  1. patch와 kernel을 원소별 곱 (element-wise multiplication)
  2. 모든 값을 합 (sum)

- **결과**
  - 스칼라 값 1개
  - 이것이 **한 필터가 한 위치에서 생성하는 출력 값**

---

### (4) Output depth (필터 수)
- 필터가 여러 개라면:
  - 같은 patch에 대해
  - **필터마다 스칼라 1개씩 생성**

- **의미**
  - 필터 개수 = 출력 depth
  - 한 공간 위치의 출력은 **벡터**가 된다.

---

### (5) Output feature map
- 모든 공간 위치에서 위 과정을 반복
- 최종 출력 텐서의 형태:
$$
(\text{Output depth},\ \text{Output height},\ \text{Output width})
$$

- 그림 맨 아래의 큐브가 바로 이 **출력 feature map**

---




<img src='https://deeplearningwithpython.io/images/ch08/how_convolution_works.fb611af4.png' width=500 height=600>

## 합성곱의 행렬식(내적) 표현

출력 feature map의 한 위치에서의 출력은  
입력 patch와 필터의 **벡터 내적(dot product)** 으로 표현할 수 있다.

### (1) 입력 patch 벡터화
공간 위치 \((h,w)\)에서의 입력 patch를 벡터로 펼치면

$$
\mathbf{x}_{h,w}
=
\mathrm{vec}\!\left(
X[:,\,h:h+3,\,w:w+3]
\right)
\in \mathbb{R}^{9C}
$$

---

### (2) 필터 벡터화
\(k\)번째 필터 역시 동일하게 벡터화하면

$$
\mathbf{w}_k
=
\mathrm{vec}\!\left(
W_k
\right)
\in \mathbb{R}^{9C}
$$

---

### (3) 출력 계산 (내적)
그러면 출력은 다음과 같이 **한 줄로 표현된다**.

$$
Y_{k,h,w}
=
\mathbf{w}_k^{\top}\mathbf{x}_{h,w}
$$

---

### (4) 모든 필터를 동시에 쓰면 (행렬 표현)

$$
\mathbf{y}_{h,w}
=
W
\mathbf{x}_{h,w},
\quad
W \in \mathbb{R}^{K \times 9C}
$$

$$\(K\): 필터 개수 (= 출력 depth)$$
$$\(\mathbf{y}_{h,w} \in \mathbb{R}^{K}\): 한 위치의 출력 벡터$$


### **1️⃣ Loss Function을 Optimization하는 W를 찾는 것이다. CNN과 FNN은 결국 같은 거네!!**


## **그림의 화살표를 코드(Numpy)**

In [24]:
import numpy as np

# 입력: (C, H, W)
C, H, W = 2, 3, 3
X = np.arange(C * H * W).reshape(C, H, W)

# 커널: (K, C, KH, KW)
# 커널의 개수(K)를 변경하면 출력 Y가 어떻게 변하나 관찰하라
K, KH, KW = 3, 3, 3
W = np.random.randn(K, C, KH, KW)

# 출력 크기 (valid convolution)
out_h = H - KH + 1
out_w = W.shape[3] - KW + 1  # 여기서는 1

Y = np.zeros((K, out_h, out_w))

for k in range(K):          # 출력 depth
    for i in range(out_h):  # height
        for j in range(out_w):  # width
            patch = X[:, i:i+KH, j:j+KW]
            Y[k, i, j] = np.sum(patch * W[k])

print("Input shape :", X.shape)
print("Kernel shape:", W.shape)
print("Output shape:", Y.shape)
print(Y)

Input shape : (2, 3, 3)
Kernel shape: (3, 2, 3, 3)
Output shape: (3, 1, 1)
[[[17.04002911]]

 [[18.15824229]]

 [[69.21007722]]]


## **그림의 화살표를 코드(Pytorch)**

In [25]:
import torch
import torch.nn.functional as F

# 입력: (N, C, H, W)
x = torch.randn(1, 2, 3, 3)

# 커널: (out_channels, in_channels, KH, KW)
conv = torch.nn.Conv2d(
    in_channels=2,
    out_channels=3,
    kernel_size=3,
    stride=1,
    padding=0,
    bias=False
)

y = conv(x)

print("Input :", x.shape)
print("Weight:", conv.weight.shape)
print("Output:", y.shape)


Input : torch.Size([1, 2, 3, 3])
Weight: torch.Size([3, 2, 3, 3])
Output: torch.Size([1, 3, 1, 1])


## **전체 그림을 코드로**

In [9]:
import torch
import torch.nn.functional as F

# 1. 입력 설정 (사용자 수정 사항 반영: 깊이 2, 높이 5, 너비 5)
# 형태: (Batch, Input_Depth, Height, Width)
input_feature_map = torch.randn(1, 2, 5, 5)
input_feature_map

tensor([[[[-0.3341,  0.8027, -0.0426,  1.1519,  0.1143],
          [-0.5779, -0.2514,  1.3043,  0.4465, -0.5680],
          [-0.2500, -0.1110, -0.2226,  0.0163, -0.2176],
          [-0.3606,  0.3649, -0.3995, -0.7237, -0.1783],
          [ 0.8111, -0.3321, -0.8809,  0.4353, -0.3596]],

         [[-0.4889, -0.0274, -0.2527,  0.8555, -0.3563],
          [-0.7299,  1.3176,  1.7900, -1.3003,  0.9683],
          [ 1.0669,  0.1300,  1.1146, -0.4801, -2.3202],
          [-0.3808, -0.7666, -1.3572,  0.5567,  0.0433],
          [ 0.3755, -1.9188,  0.8362, -1.1846, -2.8028]]]])

In [20]:
# 2. 커널(필터) 설정
# 패치 크기는 3x3이며, 출력 깊이는 그림의 변환된 패치 개수인 3으로 설정합니다.
# 형태: (Output_Depth, Input_Depth, K_Height, K_Width)
kernel_size = 3
output_depth = 3 # Kernel의 개수
kernel = torch.randn(output_depth, 2, kernel_size, kernel_size)
kernel

tensor([[[[ 1.0915e+00,  2.9470e-01,  6.4512e-01],
          [ 1.4929e+00, -6.2814e-01, -3.1293e-01],
          [ 1.2863e+00, -5.5805e-01, -1.2078e+00]],

         [[-5.2150e-01,  5.9452e-01, -4.3669e-01],
          [ 3.8306e-02, -1.8620e+00, -7.1169e-01],
          [ 1.3847e+00,  1.6417e-01, -9.6449e-01]]],


        [[[-4.8900e-01, -1.1478e-01,  2.4139e-03],
          [ 1.3722e+00,  1.0855e+00, -7.6702e-01],
          [-2.3383e-03, -5.1157e-01, -1.1957e-01]],

         [[ 6.9023e-01, -7.1949e-01, -3.5893e-01],
          [-1.6230e+00, -1.6541e-01,  1.0252e+00],
          [ 6.9860e-01, -1.9099e+00,  5.4211e-01]]],


        [[[-1.7396e+00, -1.1381e+00,  9.7556e-01],
          [ 4.7865e-01,  1.1335e+00,  3.9852e-01],
          [ 2.5051e+00, -3.5452e-01,  1.1039e+00]],

         [[-1.0863e+00, -1.4261e+00,  3.0019e-01],
          [-2.2834e-03, -4.3362e-01,  4.3319e-01],
          [-7.0888e-01, -6.0418e-01,  3.7752e-03]]]])

In [11]:
print(f"입력 특성 맵 크기: {input_feature_map.shape}") # (1, 2, 5, 5)
print(f"커널 크기: {kernel.shape}") # (3, 2, 3, 3)

입력 특성 맵 크기: torch.Size([1, 2, 5, 5])
커널 크기: torch.Size([3, 2, 3, 3])


In [13]:
# 3. 합성곱 연산 수행 (슬라이딩 윈도우 방식)
output_feature_map = F.conv2d(input_feature_map, kernel, stride=1, padding=0)
output_feature_map

tensor([[[[ 0.2780, -0.9622, -0.4993],
          [ 1.6874,  1.8505, -6.4628],
          [-0.5190,  0.5370, -2.7251]],

         [[ 3.2989,  0.4210, -3.8706],
          [ 0.3631,  4.0699,  6.5604],
          [ 3.0359,  0.9784, -4.7464]],

         [[-2.2701, -1.7041, -2.5738],
          [-2.5820,  2.3110,  4.3447],
          [ 6.5925, -0.0583, -0.0486]]]])

In [14]:
output_feature_map.squeeze()

tensor([[[ 0.2780, -0.9622, -0.4993],
         [ 1.6874,  1.8505, -6.4628],
         [-0.5190,  0.5370, -2.7251]],

        [[ 3.2989,  0.4210, -3.8706],
         [ 0.3631,  4.0699,  6.5604],
         [ 3.0359,  0.9784, -4.7464]],

        [[-2.2701, -1.7041, -2.5738],
         [-2.5820,  2.3110,  4.3447],
         [ 6.5925, -0.0583, -0.0486]]])

In [19]:
print("\n--- 연산 결과 ---")
print(f"출력 특성 맵 크기: {output_feature_map.shape}")
print(f'력 특성 맵 실제 크기{output_feature_map.squeeze().shape}')
# 결과는 (1, 3, 3, 3)이 됩니다.
# 계산식: (5 - 3) + 1 = 3


--- 연산 결과 ---
출력 특성 맵 크기: torch.Size([1, 3, 3, 3])
력 특성 맵 실제 크기torch.Size([3, 3, 3])


In [6]:
# 첫 번째 필터의 결과만 추출 (1, 1, 3, 3)
single_map = output_feature_map[:, 0:1, :, :]

# squeeze()를 사용하여 (3, 3) 크기로 압축하여 시각화 가능하게 변경
visual_ready = single_map.squeeze()
print(f"시각화용 차원: {visual_ready.shape}") # torch.Size([3, 3])

시각화용 차원: torch.Size([3, 3])
